<a href="https://colab.research.google.com/github/Reza-Navazani/Call_analysis/blob/main/call_center_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade --quiet google-cloud-speech ipywebrtc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 4.6 MB/s eta 0:00:00


In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()


In [4]:
import json
import os
import re

from IPython.display import HTML, Audio, display
from typing import Generator
from google.api_core.client_options import ClientOptions
from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech
from ipywebrtc import AudioRecorder, CameraStream

In [5]:
# fmt: off
PROJECT_ID = "big-axiom-457001-f3"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

STT_LOCATION = "us"  # @param {type: "string"}

In [6]:
! gcloud config set project {PROJECT_ID}
! gcloud auth application-default login -q
! gcloud auth application-default set-quota-project {PROJECT_ID}

Updated property [core/project].
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=u0ayarmmNHUbQBmL6vWbEiUeG3ZUNx&prompt=consent&token_usage=remote&access_type=offline&code_challenge=0QRGsXQim_jfK0Xqa3OtN4v9y-yCa5-TTWRF09KBMHo&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0Ab32j91S2xhqwgxS9TihZHHAUTV-Kp1sJwJM203HMzv--vlT1-aGwNan8p55JnOzz4mvzA

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Applicat

In [5]:
! gcloud auth application-default set-quota-project {PROJECT_ID}

ERROR: (gcloud.auth.application-default.set-quota-project) Application default credentials have not been set up. Run $ gcloud auth application-default login to set it up first.


In [7]:
client = SpeechClient(
    client_options=ClientOptions(api_endpoint=f"{STT_LOCATION}-speech.googleapis.com")
)
recognizer = client.recognizer_path(PROJECT_ID, STT_LOCATION, "_")
model = "chirp_3"
# Set a timeout for the batch recognition operation
MAX_AUDIO_LENGTH_SECS = 8 * 60 * 60

In [8]:
def print_transcript(results: list[cloud_speech.SpeechRecognitionResult]) -> None:
    for result in results:
        display(
            HTML(
                f"""<div style="word-break: break-all;">{result.alternatives[0].transcript}</div>"""
            )
        )


def generate_audio_chunks(
    audio_content: bytes, chunk_size: int
) -> Generator[bytes, None, None]:
    """Splits a byte string (like audio data) into smaller, equal-sized chunks.

    Args:
        audio_content: The raw byte data of the audio.
        chunk_size: The desired size for each audio chunk in bytes.

    Yields:
        A series of audio chunks.
    """
    # Loop through the audio content, stepping by chunk_size each time
    for start_index in range(0, len(audio_content), chunk_size):
        # The end_index is the start_index plus the chunk size
        end_index = start_index + chunk_size
        # Yield the slice of audio data for the current chunk
        yield audio_content[start_index:end_index]


def group_utterances_by_speaker_from_file(json_file_path: str) -> dict:
    """Reads a JSON file containing transcribed words and groups them into sentences spoken by each speaker."""
    with open(json_file_path, encoding="utf-8") as f:
        json_data_string = f.read()
    words_regex = r'"words":\s*(\[.*?\])'
    match = re.search(words_regex, json_data_string, re.DOTALL)

    words_list = json.loads(match.group(1))
    dialogue = []
    current_speaker = None
    current_utterance_words = []
    current_speaker = words_list[0]["speakerLabel"]

    for item in words_list:
        word = item["word"]
        speaker = item["speakerLabel"]
        # Check if the speaker has changed
        if speaker != current_speaker:
            dialogue.append(
                {"speaker": current_speaker, "text": " ".join(current_utterance_words)}
            )
            # Start a new utterance
            current_speaker = speaker
            current_utterance_words = [word]
        else:
            # Continue the current utterance
            current_utterance_words.append(word)
    # Add the final pending utterance
    if current_speaker is not None:
        dialogue.append(
            {"speaker": current_speaker, "text": " ".join(current_utterance_words)}
        )

    return {"dialogue": dialogue}

In [14]:
from google.colab import files

# Open a file-picker dialog to upload from your local machine
uploaded = files.upload()

# Example: if you uploaded "call_recording.mp3"
filename = next(iter(uploaded.keys()))
print("Uploaded file:", filename)

# You can now read the file, move it, or process it
# For example, checking its size
import os
print("File size (bytes):", os.path.getsize(filename))


Saving audio_summary_clean_energy_short.mp3 to audio_summary_clean_energy_short (1).mp3
Uploaded file: audio_summary_clean_energy_short (1).mp3
File size (bytes): 1190949


In [15]:
#/content/audio_summary_clean_energy_short.mp3
#audio_url = filename
#audio_filename = os.path.basename(audio_url)
assert os.path.exists(filename), f"File not found: {filename}"
#! wget {audio_url} -O {audio_filename}

display(Audio(filename=filename))

In [24]:

config = cloud_speech.RecognitionConfig(
    auto_decoding_config=cloud_speech.AutoDetectDecodingConfig(),
    model=model,
    language_codes=["en-US"],
)

with open(audio_filename, "rb") as f:
    audio_content = f.read()

request = cloud_speech.RecognizeRequest(
    recognizer=recognizer,
    config=config,
    content=audio_content,
)

transcript_response = client.recognize(request=request)

print_transcript(transcript_response.results)

In [22]:
from google.colab import userdata

try:
    GOOGLE_API_KEY = userdata.get('google')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [12]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [13]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [14]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [31]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),

    instruction="You are a helpful assistant which rate the procided transcription and give a score and summerise it in one sentence",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


In [32]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [38]:
transcript_text = transcript_response.results

response = await runner.run_debug(
str(transcript_text))


 ### Continue session: debug_session_id

User > [alternatives {
  transcript: "Google\'s offices and data centers around the world run on electricity, and we\'re determined to power them with carbon-free energy sources every hour of every day. Wind and solar power have helped us meet the majority of our global electricity demand. But wind turbines only spin when the wind is blowing, and solar panels only generate power when the sun is shining. So, we made an early bet on enhanced geothermal technology, which has the potential to give us access to always-on carbon-free energy. Since 2021, we\'ve worked with Fervo Energy to build a first-of-its-kind commercial project in Northern Nevada. The project started with a well drilled 8,000 ft below the surface and then extended horizontally. To access more heat beneath the ground, Fervo created fractures in the rock and drilled a second horizontal well to intersect them. Fervo then pumps cold water underground through the fractures into the se